## StyleGan Encoding for Children's Drawings

- Using NVIDIA StyleGan: https://github.com/NVlabs/stylegan
- Fine-tuning code: 
- Encoding code: https://github.com/Puzer/stylegan-encoder
  - GOT https://github.com/iyaja/stylegan-encoder

Assumes you have already done this setup and training [stylegan-finetune.ipynb](stylegan-finetune.ipynb)

In [1]:
# !pip install ipywidgets

In [2]:
# !jupyter nbextension enable --py widgetsnbextension

In [3]:
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [4]:
# !jupyter labextension list

## Encoding Setup

In [5]:
# !git clone https://github.com/iyaja/stylegan-encoder

Use Augmentor to pre-process input drawings to be square (512x512):

In [6]:
!pip install Augmentor

### Preprocess inputs

Use Augmentor to pre-process input drawings to be square (512x512):

In [7]:
pwd

'/home/jovyan/child-art'

In [8]:
import Augmentor

In [9]:
def preprocess_inputs():
    
    source = "/home/jovyan/child-art/data/to_encode"
    dest = "/home/jovyan/child-art/data/square"
    
    p = Augmentor.Pipeline(source, dest, save_format="JPEG")
    p.resize(probability=1.0, width=512, height=512)
    p.process()

In [10]:
# preprocess_inputs()

## Encoding into Latent Vector

here is a reference https://github.com/iyaja/stylegan-encoder

In [11]:
cd stylegan-encoder

/home/jovyan/child-art/stylegan-encoder


In [12]:
pwd

'/home/jovyan/child-art/stylegan-encoder'

the old way:

In [13]:
# !python encode_images.py ../data/square/ generated_images/ latent_representations/

do it inline here

In [14]:
import os
import argparse
import pickle
from tqdm import tqdm_notebook as tqdm
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator
from encoder.perceptual_model import PerceptualModel

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

/home/jovyan/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorboard/compat/ten

In [15]:
def split_to_batches(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [16]:
# paths
src_dir = "../data/square/"
generated_images_dir = "../data/generated_images/"
dlatent_dir = "../data/latent_representations/"

# training params
batch_size = 1

# perceptual model params
image_size = 256
# image_size = 512
lr = 1.0
# lr = 4.0
# iterations = 500
# iterations = 1000
# iterations = 5000
# iterations = 8000
iterations = 10000
# iterations = 20000

# generator params
randomize_noise = False#True

In [17]:
ref_images = [os.path.join(src_dir, x) for x in os.listdir(src_dir)]
ref_images = list(filter(os.path.isfile, ref_images))

if len(ref_images) == 0:
    raise Exception('%s is empty' % src_dir)

os.makedirs(generated_images_dir, exist_ok=True)
os.makedirs(dlatent_dir, exist_ok=True)

In [18]:
# URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
# URL_FFHQ = 'https://s3-us-west-2.amazonaws.com/nanonets/blogs/karras2019stylegan-ffhq-1024x1024.pkl'


# Initialize generator and perceptual model
tflib.init_tf()

# Flicker HQ not what I want
# with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
#     generator_network, discriminator_network, Gs_network = pickle.load(f)

# Use my finetuned children's drawings stylegan
model = "../stylegan/results/00021-sgan-custom-2gpu/network-snapshot-011545.pkl"
with open(model, 'rb') as f:
#     _G, _D, Gs = pickle.load(f)
    _G, _D, Gs_network = pickle.load(f)

vgg_layer = 8
# vgg_layer = 9

    
generator = Generator(Gs_network, batch_size, randomize_noise=randomize_noise)
perceptual_model = PerceptualModel(image_size, layer=vgg_layer, batch_size=batch_size)
perceptual_model.build_perceptual_model(generator.generated_image)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [19]:
# %%capture

# Optimize (only) dlatents by minimizing perceptual loss between reference and generated images in feature space
for images_batch in tqdm(split_to_batches(ref_images, batch_size), total=len(ref_images)//batch_size):
    names = [os.path.splitext(os.path.basename(x))[0] for x in images_batch]

    perceptual_model.set_reference_images(images_batch)
    op = perceptual_model.optimize(generator.dlatent_variable, iterations=iterations, learning_rate=lr)

    pbar = tqdm(op, leave=False, total=iterations)
    for loss in pbar:
        pbar.set_description(' '.join(names)+' Loss: %.2f' % loss)
    print(' '.join(names), ' loss:', loss)

    # Generate images from found dlatents and save them
    generated_images = generator.generate_images()
    generated_dlatents = generator.get_dlatents()
    for img_array, dlatent, img_name in zip(generated_images, generated_dlatents, names):
        img = PIL.Image.fromarray(img_array, 'RGB')
        img.save(os.path.join(generated_images_dir, f'{img_name}_{lr}_{iterations}.png'), 'PNG')
        np.save(os.path.join(generated_images_dir, f'{img_name}_{lr}_{iterations}.npy'), dlatent)
        np.savetxt(os.path.join(generated_images_dir, f'{img_name}_{lr}_{iterations}.txt'), dlatent, delimiter=',')
#         print(dlatent.shape, dlatent)

    generator.reset_dlatents()


to_encode_original_jasper_marker_bw.jpg_7051fc42-627a-4bca-ba0f-b4e495ddd17f  loss: 2.1590502


to_encode_original_jasper_giraffe_dragon_food_bw.jpg_8a49d328-7030-4320-806a-10b54cbdfdbe  loss: 0.6276486

